In [1]:
import glob #to read the files
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.interpolate import interp1d
import os.path 
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
from scipy.optimize import fmin

In [2]:
#kernel = [0.5 * RBF(length_scale=5, length_scale_bounds=(1, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=20, length_scale_bounds=(1, 30)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=1, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=15, length_scale_bounds=(10, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5))]

In [3]:
kernel = 0.5 * (RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)))

In [4]:
X_ = np.linspace(-10, 50, 60 + 1)
#xmin = np.linspace(-5, 5, 3)
wave = np.linspace(3000, 9000, 200 + 1)
save_path = '/home/joao/Documents/'

In [5]:
file = open("treinamento.txt")
lines = file.readlines()
file.close()

In [6]:
training_set = []

In [7]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])

In [8]:
training_set = ['SN2012fr']

In [9]:
for j in range(0,len(training_set)):
    
    list_data = [[[],[]] for y in range(0,201)]
    
    sn_names=glob.glob("/home/joao/Documents/pesquisa/templates/data_ext/"+training_set[j]+"*"+".dat")
    
    
    
    for p in range(0,len(sn_names)):

        sn_names[p] = sn_names[p][49:]
    
    #print(sn_names)
    
    print(training_set[j])
    
    #print(sn_names)
    
    if len(sn_names) > 4:
        
        for i in range(0,len(sn_names)):

            #y = []
            z = []

            file = open(sn_names[i])
            lines = file.readlines()
            file.close()

            for line in lines:
                if line.startswith('#'): continue
                co=line.rstrip().replace('INDEF','Nan').split()

                #y.append(co[0])
                z.append(co[1])
        

            #y = np.array(y, dtype=float)
            z = np.array(z, dtype=float)
            #print(z)
            
            
            for l in range(0,len(z)):
                
                if z[l] < 10**(-64):
                    
                     z[l] = 0
            
            
            timetemp = lines[0]
            timetemp = timetemp[:-1]
            timetemp = float(timetemp[1:])
            
            for jj in range(0,len(z)):
                
                list_data[jj][0].append([timetemp])
                list_data[jj][1].append(z[jj])

            #print(i)
            
        Export = []
        Export_cov = []

        for jj in range(0,len(list_data)):
            
            X_mod = []
            
            Y_mean_mod = []
            
            count = 0
            
            for jjj in range(0,len(list_data[jj][1])):
                
                if list_data[jj][1][jjj] > 10**(-64):
                    
                    count += 1        
                    
            #print(count/len(list_data[jj][1]))
            
            if count > 4:
                
                
                
                for ii in range(0,len(list_data[jj][1])):
                    
                    if list_data[jj][1][ii] > 10**(-64):
                        
                        Y_mean_mod.append(list_data[jj][1][ii])
                        X_mod.append(list_data[jj][0][ii])
                    
                    

                #option = []
                #score = []  

                #print(list_data[jj][1])
                ff = max(Y_mean_mod)
                #print(ff)
                Y_mean_mod = Y_mean_mod/ff

                gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(X_mod, Y_mean_mod)
                y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)        
                
                #print(y_cov)

                for l in range(0,len(y_mean)):

                    Export.append(y_mean[l]*ff)
                    Export_cov.append(np.sqrt(np.diag(y_cov))[l]*ff)


            else:

                y_meaneff = [0 for y in range(0,61)]
                #print(len(y_mean), len(X_))

                for l in range(0,len(y_meaneff)):

                    Export.append(y_meaneff[l])
                    Export_cov.append(0)
             

        table = []
        for l in range(0,len(Export)):
            #table.append((Export[l],training_set[j]))
            table.append((Export[l], Export_cov[l]))

        name_of_file = training_set[j]

        completeName = os.path.join(save_path, name_of_file+"_diss.dat")

        f = open(completeName, 'w')
        f.write(tabulate(table, tablefmt="plain"))
        f.close()


SN2012fr


In [10]:
y_cov

array([[ 7.44688839e-03,  3.39286803e-03,  2.27338496e-03, ...,
        -1.85797967e-06, -6.29313122e-06, -9.28093236e-06],
       [ 3.39286803e-03,  5.38892469e-03,  1.85269224e-03, ...,
         3.99189261e-07, -2.45394144e-06, -4.57078020e-06],
       [ 2.27338496e-03,  1.85269224e-03,  4.24314260e-03, ...,
         1.76981330e-06,  4.92854385e-07, -6.48939560e-07],
       ...,
       [-1.85797967e-06,  3.99189261e-07,  1.76981330e-06, ...,
         5.05664668e-03,  2.20992428e-03,  2.08181762e-03],
       [-6.29313122e-06, -2.45394144e-06,  4.92854385e-07, ...,
         2.20992428e-03,  5.05965943e-03,  2.22619505e-03],
       [-9.28093236e-06, -4.57078020e-06, -6.48939560e-07, ...,
         2.08181762e-03,  2.22619505e-03,  5.08403674e-03]])